In [9]:
# Install dependencies (if needed)
# We need langgraph-checkpoint-sqlite for SqliteSaver
%pip install -qU langgraph langchain langchain_openai python-dotenv langgraph-checkpoint-sqlite

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

# If the key is not in .env, ask for input (useful in labs)
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = input("Enter your OpenAI API Key: ")

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
print("LLM initialized!")

LLM initialized!


### Exercise 1: InMemorySaver — Runtime-only Memory

In this exercise, we will create a simple graph with **in-memory** persistence using `InMemorySaver`.

Objectives:
- Define an `AgentState` with `messages: Annotated[list, operator.add]`.
- Create a `chatbot` node that uses the LLM to respond to the user.
- Use `InMemorySaver` as the checkpointer.
- Invoke the graph twice with the **same** `thread_id` and observe that the message history grows.

In [15]:
from typing import TypedDict, Annotated
import operator

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage


class AgentState(TypedDict):
    # operator.add -> concatenates message lists between nodes
    messages: Annotated[list, operator.add]


def chatbot(state: AgentState) -> dict:
    """Main node that calls the LLM and adds a new message to the history."""
    response = llm.invoke(state["messages"])
    return {"messages": [response]}


# Graph Builder
builder = StateGraph(AgentState)
builder.add_node("chatbot", chatbot)
builder.set_entry_point("chatbot")
builder.add_edge("chatbot", END)

# In-memory Checkpointer (does not persist to disk)
memory = InMemorySaver()
graph_in_memory = builder.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "lab-thread-1"}}

print("--- First Call ---")
result1 = graph_in_memory.invoke({"messages": [HumanMessage(content="My name is Carla.")]}, config=config)
for m in result1["messages"]:
    print(m.type, ":", m.content)

print("\n--- Second Call (same thread) ---")
result2 = graph_in_memory.invoke({"messages": [HumanMessage(content="Do you remember my name?")]}, config=config)
for m in result2["messages"]:
    print(m.type, ":", m.content)

--- First Call ---
human : My name is Carla.
ai : Hello, Carla! How can I assist you today?

--- Second Call (same thread) ---
human : My name is Carla.
ai : Hello, Carla! How can I assist you today?

--- Second Call (same thread) ---
human : My name is Carla.
ai : Hello, Carla! How can I assist you today?
human : Do you remember my name?
ai : Yes, your name is Carla. How can I help you today?
human : My name is Carla.
ai : Hello, Carla! How can I assist you today?
human : Do you remember my name?
ai : Yes, your name is Carla. How can I help you today?


### Exercise 2: SqliteSaver — Persistent Memory across Sessions

Now let's swap `InMemorySaver` for `SqliteSaver` so that the graph state is saved to disk.

Objectives:
- Create a `SqliteSaver` pointing to a `lab_memory.sqlite` file.
- Compile the graph with this checkpointer.
- Invoke the graph multiple times with the same `thread_id` and confirm that memory is retained even if the kernel is restarted.

In [16]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

# Connect (or create) the SQLite database for the lab
conn = sqlite3.connect("lab_memory.sqlite", check_same_thread=False)
sqlite_checkpointer = SqliteSaver(conn)

graph_sqlite = builder.compile(checkpointer=sqlite_checkpointer)

config_alice = {"configurable": {"thread_id": "user-alice"}}
config_bob = {"configurable": {"thread_id": "user-bob"}}

print("--- Conversation with Alice (thread: user-alice) ---")
res1 = graph_sqlite.invoke({"messages": [HumanMessage(content="My name is Alice.")]}, config=config_alice)
res2 = graph_sqlite.invoke({"messages": [HumanMessage(content="What is my name?")]}, config=config_alice)

for m in res2["messages"]:
    print(m.type, ":", m.content)

print("\n--- Conversation with Bob (thread: user-bob) ---")
res3 = graph_sqlite.invoke({"messages": [HumanMessage(content="My name is Bob.")]}, config=config_bob)
res4 = graph_sqlite.invoke({"messages": [HumanMessage(content="Do you remember my name?")]}, config=config_bob)

for m in res4["messages"]:
    print(m.type, ":", m.content)

--- Conversation with Alice (thread: user-alice) ---
human : My name is Alice.
ai : Hello, Alice! How can I assist you today?
human : What is my name?
human : Wait, I changed my mind. My name is Alice Cooper.
ai : Got it! Your name is Alice Cooper. How can I help you today, Alice Cooper?
human : My name is Alice.
ai : Thanks for letting me know, Alice! How can I assist you today?
human : What is my name?
ai : You mentioned that your name is Alice. How can I assist you further, Alice?

--- Conversation with Bob (thread: user-bob) ---
human : My name is Alice.
ai : Hello, Alice! How can I assist you today?
human : What is my name?
human : Wait, I changed my mind. My name is Alice Cooper.
ai : Got it! Your name is Alice Cooper. How can I help you today, Alice Cooper?
human : My name is Alice.
ai : Thanks for letting me know, Alice! How can I assist you today?
human : What is my name?
ai : You mentioned that your name is Alice. How can I assist you further, Alice?

--- Conversation with Bo

### Exercise 3: Time Travel — Inspecting Checkpoints

Finally, let's inspect the checkpoints saved in SQLite.

Objectives:
- List checkpoints for a `thread_id` using `SqliteSaver`.
- Get the last saved state via `graph_sqlite.get_state(config)`.
- Discuss how to implement an "undo last action" by reverting to a previous checkpoint.

In [17]:
# List state history (checkpoints)
# The get_state_history method returns a generator with reverse history (newest to oldest)
# We need to use the correct config (config_alice) defined in the previous cell
history = list(graph_sqlite.get_state_history(config_alice))
print(f"Total snapshots in history: {len(history)}")

for i, snapshot in enumerate(history[:3]):  # Show the last 3
    print(f"\n--- Snapshot {i} (ID: {snapshot.config['configurable']['checkpoint_id']}) ---")
    print("Messages:", snapshot.values.get("messages", []))
    print("Next step:", snapshot.next)

# TIME TRAVEL: Let's load an old state!
# We take the second to last state (before the bot's answer, for example)
if len(history) >= 2:
    past_snapshot = history[1] # Second to last state
    past_config = past_snapshot.config
    
    print(f"\n--- TIME TRAVELING to checkpoint {past_config['configurable']['checkpoint_id']} ---")
    # By invoking the graph with a config containing 'checkpoint_id', it loads that state
    # Let's add a NEW message from this past point, creating a "parallel universe"
    
    fork_config = {
        "configurable": {
            "thread_id": "user-alice",
            "checkpoint_id": past_config['configurable']['checkpoint_id']
        }
    }
    
    print("Adding message at a past point...")
    # Note: This creates a new fork in history!
    res_fork = graph_sqlite.invoke(
        {"messages": [HumanMessage(content="Wait, I changed my mind. My name is Alice Cooper.")]}, 
        config=fork_config
    )
    
    print("\nNew bot response in the parallel universe:")
    for m in res_fork["messages"]:
        print(m.type, ":", m.content)

Total snapshots in history: 15

--- Snapshot 0 (ID: 1f0db9b8-3913-61a0-800c-53ef3a31506f) ---
Messages: [HumanMessage(content='My name is Alice.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello, Alice! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 12, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_d0c93c37b1', 'id': 'chatcmpl-CnuhxKnjCSmgziQvJkzioPqmbwyMe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d92d660b-e619-415a-8241-618da61dcdae-0', usage_metadata={'input_tokens': 12, 'output_tokens': 11, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details